# Load Train Set

features (3681) | labels (1)

In [2]:
from lib.data import load_dataset
x_train, y_train, x_test, y_test = load_dataset('dts/plain_dataset.csv')

In [2]:
# import pandas as pd
import cudf
import cupy
import pandas as pd
import numpy as np

# df = pd.read_feather("dts/np_dataset_train.ftr")
df = pd.read_csv("dts/np_dataset_train.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 3684 entries, Unnamed: 0 to Label
dtypes: bool(1), float64(3682), int64(1)
memory usage: 1.1 GB


## train/test Split

In [5]:
from sklearn.model_selection import train_test_split

rs = 1

train, test = train_test_split(df, test_size=0.2, random_state=rs)
train.info()
test.info()

x_train = pd.DataFrame(train.iloc[1:, :-2])
x_test  = pd.DataFrame(test.iloc[1:, :-2])
y_train = np.array(train.iloc[1:, -1])
y_test  = np.array(test.iloc[1:, -1])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32000 entries, 21721 to 33003
Columns: 3684 entries, Unnamed: 0 to Label
dtypes: bool(1), float64(3682), int64(1)
memory usage: 899.4 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 3841 to 32191
Columns: 3684 entries, Unnamed: 0 to Label
dtypes: bool(1), float64(3682), int64(1)
memory usage: 224.9 MB


## Classifier definition

In [ ]:
from random import seed
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier as tree_classifier
from scipy import stats
from math import sqrt

class random_forest():
    def __init__(self, number_of_trees=1, number_of_iteractions=1, max_depth=1, random_state=1):
        self.iteractions = number_of_iteractions
        self.number_of_trees = number_of_trees
        self.max_depth = max_depth
        self.forest = []
        self.random_state = random_state

    def bootstrap(self, x_samples):
        """ Create subsets of the input, and returns the indexes 
            from the original sample."""
        index = []

        np.random.RandomState(self.random_state)
        sample_length = len(x_samples)
        for i in range(self.number_of_trees):
            index.append(np.random.randint(0, sample_length, [
                         1, round(sqrt(sample_length))]))
        return index

    def predict(self, X):
        y = []
        for tree in self.forest:
            y.append(tree.predict(X))
        mode_ = stats.mode(y, axis=0)
        mode_ = np.transpose(mode_[0])
        return mode_

    def fit(self, x_samples, y_samples):
        idx = self.bootstrap(y_samples)
        for t in range(self.number_of_trees):
            tree = tree_classifier(
                criterion="gini", max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(x_samples.iloc[(flatten(idx[t]))[0],:], y_samples[(flatten(idx[t]))[0]])
            self.forest.append(tree)

    # def train(self, x_samples, y_samples):
    #     self.bagging(x_samples, y_samples)


def flatten(indices):
    return [item for item in indices]


In [ ]:
# from sklearn import svm
# from cuml.svm import SVC
# import random_forest as rf

clf = random_forest(number_of_trees=1, number_of_iteractions=1, max_depth=1)


## Sending Data to GPU

In [ ]:
x = pd.DataFrame(train.iloc[:, :-2])
y = np.array(train.iloc[:, -1])

## Model Fitting

In [ ]:
clf.fit(x, y)

## Testing

In [ ]:
y_pred = clf.predict(test.iloc[:,:-2])

## Evaluating

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score



accuracy = accuracy_score(y_true=test.iloc[:, -1], y_pred=y_pred)
roc = roc_auc_score(test.iloc[:, -1], y_pred)
pre = precision_score(y_true=test.iloc[:, -1], y_pred=y_pred)
rec = recall_score(y_true=test.iloc[:, -1], y_pred=y_pred)


print("Metrics: \n Acc: {acc:.2f}, ROC: {roc:.2f}\n PRE: {pre:.2f}, REC: {rec:.2f}".format(acc=accuracy,roc=roc,pre=pre,rec=rec))


Metrics: 
 Acc: 1.00, ROC: 1.00
 PRE: 1.00, REC: 1.00


In [6]:
import time
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

importance = clf.feature_importances_

for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()


# start_time = time.time()
# result = permutation_importance(
#     clf, X=x_test, y=y_test, n_repeats=2, random_state=42, n_jobs=1
# )
# elapsed_time = time.time() - start_time
# print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

# forest_importances = pd.Series(result.importances_mean, index=feature_names)

ValueError: Number of labels=32000 does not match number of samples=31999

In [ ]:
pred = pd.DataFrame({'Id': columns,
                     'Category': y_pred})

pred.to_csv('predictions.csv', index=False)

NameError: name 'columns' is not defined

## SC graph

In [ ]:
# accs = []
# rocs = []
# pres = []
# recs = []


# for rs in range(100):
#     train, test = train_test_split(df, test_size=0.2, random_state=rs)

#     clf = SVC(random_state=rs)

#     x = cudf.DataFrame(train.iloc[:, :-1])
#     y = cupy.array(train.iloc[:, -1])

#     clf.fit(x, y)
#     y_pred = clf.predict(test.iloc[:, :-1])

#     accuracy = accuracy_score(y_true=test.iloc[:, -1], y_pred=y_pred)
#     accs.append(accuracy)
#     roc = roc_auc_score(test.iloc[:, -1], y_pred)
#     rocs.append(roc)
#     pre = precision_score(y_true=test.iloc[:, -1], y_pred=y_pred)
#     pres.append(pre)
#     rec = recall_score(y_true=test.iloc[:, -1], y_pred=y_pred)
#     recs.append(rec)

#     print("{i}/100".format(i=rs+1))

In [ ]:


# import matplotlib.pyplot as plt


# plt.stairs(accs)

# npAccs = np.array(accs)
# npRocs = np.array(rocs)
# npPres = np.array(pres)
# npRecs = np.array(recs)

# def min_max(name,array):
#     a = np.array(array)
#     print("{name}: ({min:.6f},{max:.1f}|{mean:.6f})".format(
#         name=name, min=np.min(a), max=np.max(a), mean=np.mean(a)))

# min_max("Acc",accs)
# min_max("Roc", rocs)
# min_max("Pre", pres)
# min_max("Rec", recs)


NameError: name 'accs' is not defined